# Imports

In [1]:
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.insert(0, "lib/")
from data.coco_dataset import CocoDataset
from utils.preprocess_sample import preprocess_sample
from utils.collate_custom import collate_custom
from utils.utils import to_cuda_variable
import utils.result_utils as result_utils
from utils.json_dataset_evaluator import evaluate_boxes
from model.detector import detector

torch_ver = torch.__version__[:3]

compiling/loading roi_align
Detected CUDA files, patching ldflags
Emitting ninja build file /sequoia/data1/iroccosp/GIT/bitbucket/detectorch/lib/cppcuda/build/build.ninja...
Building extension module roialign...
Loading extension module roialign...


# Parameters

In [2]:
# Pretrained model
# https://s3-us-west-2.amazonaws.com/detectron/36224046/12_2017_baselines/fast_rcnn_R-50-C4_2x.yaml.08_22_57.XFxNqEnL/output/train/coco_2014_train%3Acoco_2014_valminusminival/generalized_rcnn/model_final.pkl
arch='resnet50'
pretrained_model_file = 'files/trained_models/fast/model_final.pkl'
mapping_file = 'files/mapping_files/resnet50_mapping.npy'

# Pre-computed COCO minival2014 proposals
# https://s3-us-west-2.amazonaws.com/detectron/35998355/12_2017_baselines/rpn_R-50-C4_1x.yaml.08_00_43.njH5oD9L/output/test/coco_2014_minival/rpn/rpn_proposals.pkl
proposal_file='files/proposal_files/coco_2014_minival/rpn_proposals.pkl'

# COCO minival2014 dataset path
coco_ann_file='datasets/data/coco/annotations/instances_minival2014.json'
img_dir='datasets/data/coco/val2014'

# Create dataset

In [3]:
dataset = CocoDataset(ann_file=coco_ann_file,img_dir=img_dir,proposal_file=proposal_file,
                       sample_transform=preprocess_sample(target_sizes=[800]))
dataloader = DataLoader(dataset, batch_size=1, # only batch_size=1 is supported by now
                        shuffle=False, num_workers=4, collate_fn=collate_custom)

loading annotations into memory...
Done (t=1.17s)
creating index...
index created!


# Create detector model

In [4]:
model = detector(arch=arch,
                 caffe_pkl_file=pretrained_model_file,
                 mapping_file=mapping_file)
model = model.cuda()

# Evaluate

In [5]:
# Create data structure to store results
all_boxes, all_segms, all_keyps = result_utils.empty_results(len(dataset.classes), len(dataset)) 
# (only all_boxes will be used for fast RCNN)

In [ ]:
# Compute detections for whole dataset
for i, batch in enumerate(dataloader):
    batch = to_cuda_variable(batch)
    # forward pass
    if torch_ver=="0.4": # handle change in "volatile"
        with torch.no_grad():    
            class_scores,bbox_deltas,_,_ =model(batch['image'],batch['proposal_coords'])
    else:
        class_scores,bbox_deltas,_,_ =model(batch['image'],batch['proposal_coords'])
    # postprocess output:
    # - convert coordinates back to original image size, 
    # - treshold proposals based on score,
    # - do NMS.
    scores_final, boxes_final, boxes_per_class = result_utils.postprocess_output(batch['proposal_coords'],
                                                                    batch['scaling_factors'],
                                                                    batch['original_im_size'],
                                                                    class_scores,
                                                                    bbox_deltas)
    # store results
    result_utils.extend_results(i, all_boxes, boxes_per_class)
    
    if i%100==0:
        print("{}/{}".format(i+1,len(dataset)))

print('Done!')

In [8]:
# Save detection results
np.save('files/results/all_boxes.npy',all_boxes)

In [7]:
# Compute evaluation metrics
coco_eval = evaluate_boxes(json_dataset=dataset.coco, 
                           all_boxes=all_boxes, 
                           output_dir='files/results/',
                           use_salt=False, cleanup=False)

Loading and preparing results...
DONE (t=1.26s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=35.10s).
Accumulating evaluation results...
DONE (t=5.47s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.356
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.567
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.382
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.181
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.403
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.494
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.302
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.466
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.486
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1